# SSVEP 腦波分析完整教學
## Steady-State Visual Evoked Potential 穩態視覺誘發電位

### 營隊教學用

---

## 課程大綱
1. SSVEP 原理介紹
2. 數據載入與視覺化
3. 訊號預處理
4. 特徵提取
5. 機器學習分類
6. 結果評估

---

## 1. SSVEP 原理簡介

### 什麼是 SSVEP？
- 當我們看著特定頻率閃爍的光源時，大腦視覺皮層會產生相同頻率的電位反應
- 例如：看著 10Hz 閃爍的燈光，腦波中會出現 10Hz 的訊號
- 應用：腦機介面(BCI)、拼字器、輪椅控制等

### 常見刺激頻率：
- 8Hz, 10Hz, 12Hz, 15Hz 等
- 通常使用 4-6 個不同頻率作為控制指令


In [ ]:
# 安裝必要套件
!pip install -q mne

print("套件安裝完成")

In [ ]:
# 導入所需函式庫
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.fft import fft, fftfreq
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# 設定繪圖風格
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("所有套件載入完成")
print(f"NumPy 版本: {np.__version__}")

## 2. 產生示範資料集 SSVEP 數據

為了教學方便，我們產生示範資料集的 SSVEP 訊號

In [ ]:
# 實驗參數設定（改用 MNE 內建 SSVEP dataset）

# 目標刺激頻率（此 dataset 主要示範 12 Hz 與 15 Hz）
target_freqs = [12, 15]  # Hz
n_classes = len(target_freqs)

# Epoch 設定：每個 trial 擷取的時間窗
tmin = 0.0
tmax = 4.0  # 秒（你也可以改成 2.0 或 3.0 做比較）

print(f"實驗設定：")
print(f"  刺激頻率: {target_freqs} Hz")
print(f"  Epoch 時間窗: {tmin} ~ {tmax} 秒")


In [ ]:
# 使用 MNE 內建 SSVEP dataset 取代「模擬訊號」
# 目標：產生與原本相同格式的 X_data, y_labels
#   X_data shape: (n_trials_total, n_channels, n_samples)
#   y_labels shape: (n_trials_total,)
#
# 註：此 dataset 為 BIDS 結構，實際資料是 BrainVision 格式（.vhdr/.eeg/.vmrk）

import numpy as np
from pathlib import Path

import mne

def _find_bids_root(data_path: Path) -> Path:
    """嘗試找出 BIDS root（不同版本的 MNE 可能回傳不同層級的資料夾）。"""
    data_path = Path(data_path)
    # 常見情況 1：data_path 已經是 .../MNE-ssvep-data/ssvep
    if (data_path / "dataset_description.json").exists():
        return data_path
    # 常見情況 2：data_path 是上層，ssvep 在子資料夾
    for cand in [data_path / "ssvep", data_path / "MNE-ssvep-data" / "ssvep", data_path / "MNE-ssvep-data"]:
        if (cand / "dataset_description.json").exists():
            return cand
    # 最後 fallback：往下找第一個 dataset_description.json
    hits = list(data_path.rglob("dataset_description.json"))
    if hits:
        return hits[0].parent
    raise RuntimeError(f"找不到 BIDS root: {data_path}")

def load_mne_ssvep_as_numpy(target_freqs, tmin=0.0, tmax=4.0, picks="eeg"):
    data_path = Path(mne.datasets.ssvep.data_path())
    bids_root = _find_bids_root(data_path)

    # 找所有 BrainVision header 檔
    vhdr_files = sorted(bids_root.rglob("*.vhdr"))
    if len(vhdr_files) == 0:
        raise RuntimeError(f"在 {bids_root} 找不到 .vhdr 檔案")

    freq_to_label = {f: i for i, f in enumerate(target_freqs)}
    X_list, y_list = [], []

    last_epochs = None

    for vhdr in vhdr_files:
        raw = mne.io.read_raw_brainvision(vhdr, preload=True, verbose=False)

        # 由 annotations 產生事件
        events, event_id = mne.events_from_annotations(raw, verbose=False)

        # 依據描述字串，挑出包含目標頻率的事件
        selected = {}
        for desc, code in event_id.items():
            desc_low = desc.lower()
            for f in target_freqs:
                # 盡量涵蓋「12」「12.0」「12 hz」「12hz」等描述方式
                if (str(f) in desc_low) or (str(float(f)) in desc_low):
                    if ("hz" in desc_low) or ("ssvep" in desc_low) or ("stim" in desc_low) or ("freq" in desc_low):
                        selected[desc] = code

        # 如果描述中沒有明確的 hz 字樣，就放寬條件再試一次
        if len(selected) == 0:
            for desc, code in event_id.items():
                desc_low = desc.lower()
                for f in target_freqs:
                    if (str(f) in desc_low) or (str(float(f)) in desc_low):
                        selected[desc] = code

        if len(selected) == 0:
            # 這個檔案沒有我們要的事件就跳過
            continue

        # 建立 epochs
        epochs = mne.Epochs(
            raw,
            events,
            event_id=selected,
            tmin=tmin,
            tmax=tmax,
            baseline=None,
            picks=picks,
            preload=True,
            verbose=False,
        )
        last_epochs = epochs

        # 把不同頻率的 trials 合併成 (n_trials_total, n_channels, n_samples)
        for f in target_freqs:
            # 找對應頻率的 event key（第一個符合者）
            key = None
            for desc in selected.keys():
                d = desc.lower()
                if (str(f) in d) or (str(float(f)) in d):
                    key = desc
                    break
            if key is None:
                continue

            data = epochs[key].get_data()  # (n_trials, n_channels, n_times)
            if data.size == 0:
                continue
            X_list.append(data)
            y_list.append(np.full(data.shape[0], freq_to_label[f], dtype=int))

    if len(X_list) == 0:
        raise RuntimeError("沒有成功載入任何 trials。請確認 target_freqs 與事件標記是否匹配。")

    X_data = np.concatenate(X_list, axis=0)
    y_labels = np.concatenate(y_list, axis=0)

    # 回填與原本 notebook 一致的參數命名
    sampling_rate = float(last_epochs.info["sfreq"])
    n_channels = X_data.shape[1]
    duration = float(tmax - tmin)
    n_trials_total = X_data.shape[0]

    print("已從 MNE 內建 SSVEP dataset 載入數據")
    print(f"BIDS root: {bids_root}")
    print(f"總 trials 數: {n_trials_total}")
    print(f"數據形狀: {X_data.shape}  (trials, channels, samples)")
    print(f"標籤形狀: {y_labels.shape}")
    print(f"取樣頻率: {sampling_rate} Hz")
    print(f"通道數: {n_channels}")
    print(f"Epoch 長度: {duration} 秒")

    return X_data, y_labels, sampling_rate, duration, n_channels

X_data, y_labels, sampling_rate, duration, n_channels = load_mne_ssvep_as_numpy(
    target_freqs=target_freqs,
    tmin=tmin,
    tmax=tmax,
    picks="eeg",
)


## 3. 數據視覺化

讓我們看看 SSVEP 訊號長什麼樣子

In [ ]:
# 繪製不同頻率的原始訊號（每個頻率挑一個代表 trial）
n_rows = int(np.ceil(n_classes / 2))
n_cols = 2
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
axes = np.array(axes).reshape(-1)

time_axis = np.linspace(0, duration, X_data.shape[2])

for class_idx, freq in enumerate(target_freqs):
    # 取得該頻率的第一個 trial
    trial_idx = np.where(y_labels == class_idx)[0][0]
    trial_data = X_data[trial_idx]  # shape: (n_channels, n_samples)

    # 繪製所有通道（加上垂直位移方便觀看）
    for ch in range(n_channels):
        axes[class_idx].plot(time_axis, trial_data[ch] + ch * 3, alpha=0.7, linewidth=1)

    axes[class_idx].set_title(f'{freq} Hz Stimulus - Raw EEG Signal', fontsize=14, fontweight='bold')
    axes[class_idx].set_xlabel('Time (s)', fontsize=11)
    axes[class_idx].set_ylabel('Amplitude (+ offset)', fontsize=11)
    axes[class_idx].grid(True, alpha=0.3)

# 關掉多餘的子圖（當 n_classes 不是 2 的倍數時）
for k in range(n_classes, len(axes)):
    axes[k].axis('off')

plt.tight_layout()
plt.suptitle('SSVEP Raw Signal Visualization', fontsize=16, y=1.02)
plt.show()

print("圖說明: 每個子圖顯示不同刺激頻率下的 EEG 原始訊號")
print("注意: SSVEP 在時域不一定很明顯，通常需要看頻域特徵")


In [ ]:
# 頻譜分析：看看頻域特徵（目標頻率與二次諧波）

def compute_spectrum(signal_data, sampling_rate):
    """計算單一 trial 的功率頻譜（簡單 FFT 版本）"""
    channel_data = signal_data[-1]  # 使用最後一個通道當示範
    n = len(channel_data)
    fft_vals = fft(channel_data)
    fft_freq = fftfreq(n, 1 / sampling_rate)

    positive = fft_freq > 0
    fft_freq = fft_freq[positive]
    fft_power = np.abs(fft_vals[positive])
    return fft_freq, fft_power

n_rows = int(np.ceil(n_classes / 2))
n_cols = 2
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
axes = np.array(axes).reshape(-1)

for class_idx, freq in enumerate(target_freqs):
    # 取該頻率第一個 trial
    trial_idx = np.where(y_labels == class_idx)[0][0]
    trial_data = X_data[trial_idx]

    freqs, power = compute_spectrum(trial_data, sampling_rate)

    axes[class_idx].plot(freqs, power, linewidth=2, label="Spectrum")

    # 標示目標頻率與二次諧波位置
    axes[class_idx].axvline(freq, linestyle="--", linewidth=2, label=f"Target {freq} Hz")
    axes[class_idx].axvline(freq * 2, linestyle="--", linewidth=2, alpha=0.7, label=f"2nd Harmonic {freq*2} Hz")

    axes[class_idx].set_xlim([0, 40])
    axes[class_idx].set_title(f"Frequency Spectrum: {freq} Hz Stimulus", fontsize=14, fontweight="bold")
    axes[class_idx].set_xlabel("Frequency (Hz)", fontsize=11)
    axes[class_idx].set_ylabel("Power", fontsize=11)
    axes[class_idx].legend()
    axes[class_idx].grid(True, alpha=0.3)

for k in range(n_classes, len(axes)):
    axes[k].axis("off")

plt.tight_layout()
plt.suptitle("SSVEP Frequency Analysis - Target Frequency Peaks", fontsize=16, y=1.02)
plt.show()

print("圖說明: 這是 SSVEP 訊號的頻譜分析")
print("理想情況下，目標頻率與二次諧波附近會出現較明顯的峰值")


## 4. 訊號預處理

預處理步驟:
1. 帶通濾波 (5-40Hz) - 保留 SSVEP 相關頻段
2. 標準化

In [ ]:
def preprocess_signal(data, sampling_rate, lowcut=5, highcut=40):
    """
    預處理 SSVEP 訊號

    參數:
        data: 輸入訊號 (n_channels, n_samples)
        sampling_rate: 取樣頻率
        lowcut: 低頻截止
        highcut: 高頻截止
    """
    data = np.asarray(data)
    filtered_data = np.zeros_like(data)

    # 設計帶通濾波器
    nyquist = sampling_rate / 2
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(4, [low, high], btype='band')

    # 濾波（逐通道）
    for ch in range(data.shape[0]):
        filtered_data[ch] = signal.filtfilt(b, a, data[ch])

    # 標準化（逐通道）
    for ch in range(filtered_data.shape[0]):
        std = np.std(filtered_data[ch])
        if std < 1e-12:
            std = 1e-12
        filtered_data[ch] = (filtered_data[ch] - np.mean(filtered_data[ch])) / std

    return filtered_data

# 預處理所有數據
print("預處理數據中...")
X_preprocessed = np.array([preprocess_signal(trial, sampling_rate) for trial in X_data])

print("預處理完成")
print(f"處理後數據形狀: {X_preprocessed.shape}")


In [ ]:
# 比較預處理前後的訊號
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

sample_trial = 0
sample_channel = -1  # 最後一個通道
time_axis = np.linspace(0, duration, X_data.shape[2])

# 原始訊號
axes[0].plot(time_axis, X_data[sample_trial, sample_channel], linewidth=1)
axes[0].set_title('Before Preprocessing - Raw Signal', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Time (sec)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True, alpha=0.3)

# 預處理後訊號
axes[1].plot(time_axis, X_preprocessed[sample_trial, sample_channel], linewidth=1, color='green')
axes[1].set_title('After Preprocessing - Filtered + Normalized', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Time (sec)')
axes[1].set_ylabel('Amplitude')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("圖說明: 比較預處理前後的訊號")
print("預處理後的訊號更乾淨，雜訊被過濾掉了")

## 5. 特徵提取

我們使用功率譜密度 (Power Spectral Density, PSD) 作為特徵

方法: 計算每個目標頻率及其諧波的功率

In [ ]:
def extract_features(signal_data, sampling_rate, target_freqs, n_harmonics=2):
    """
    提取 SSVEP 特徵（以 Welch PSD 為例）

    參數:
        signal_data: 訊號 (n_channels, n_samples)
        sampling_rate: 取樣頻率
        target_freqs: 目標頻率列表
        n_harmonics: 使用的諧波數量

    回傳:
        features: 1D 特徵向量
    """
    signal_data = np.asarray(signal_data)
    features = []

    # 逐通道計算 PSD，並在目標頻率與諧波附近取 band power
    for ch in range(signal_data.shape[0]):
        freqs, psd = signal.welch(
            signal_data[ch],
            fs=sampling_rate,
            nperseg=min(1024, signal_data.shape[1]),
            noverlap=None,
            scaling="density",
        )

        for f in target_freqs:
            for h in range(1, n_harmonics + 1):
                f0 = f * h
                # 取 +/-0.5 Hz 的小頻帶平均 power
                band = (freqs >= f0 - 0.5) & (freqs <= f0 + 0.5)
                if not np.any(band):
                    power = 0.0
                else:
                    power = float(np.mean(psd[band]))
                features.append(power)

    return np.array(features, dtype=float)

# 提取所有 trial 的特徵
print("提取特徵中...")
X_features = np.array([extract_features(trial, sampling_rate, target_freqs) for trial in X_preprocessed])

print("特徵提取完成")
print(f"特徵矩陣形狀: {X_features.shape}")
print(f"每個 trial 的特徵數: {X_features.shape[1]}")


In [ ]:
# 視覺化特徵分布
n_rows = int(np.ceil(n_classes / 2))
n_cols = 2
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows + 2))
axes = np.array(axes).reshape(-1)

for class_idx, freq in enumerate(target_freqs):
    # 取得該類別的所有特徵
    class_features = X_features[y_labels == class_idx]

    # 繪製前 16 個特徵的分布
    axes[class_idx].boxplot(class_features[:, :16], labels=range(1, 17))
    axes[class_idx].set_title(f"Class {class_idx}: {freq} Hz - Feature Distribution", fontsize=13, fontweight="bold")
    axes[class_idx].set_xlabel("Feature Index", fontsize=11)
    axes[class_idx].set_ylabel("Feature Value", fontsize=11)
    axes[class_idx].tick_params(axis="x", rotation=0)
    axes[class_idx].grid(True, alpha=0.3)

for k in range(n_classes, len(axes)):
    axes[k].axis("off")

plt.tight_layout()
plt.suptitle("Feature Distribution by Class", fontsize=16, y=1.02)
plt.show()


## 6. 機器學習分類

我們使用支持向量機 (Support Vector Machine, SVM) 來辨識 SSVEP 頻率

In [ ]:
# 分割訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_labels, test_size=0.3, random_state=42, stratify=y_labels
)

print(f"訓練集大小: {X_train.shape[0]} trials")
print(f"測試集大小: {X_test.shape[0]} trials")
print(f"\n訓練集標籤分布: {np.bincount(y_train)}")
print(f"測試集標籤分布: {np.bincount(y_test)}")

In [ ]:
# 訓練 SVM 分類器
print("訓練 SVM 分類器...")

clf = SVC(kernel='rbf', C=1.0, gamma='scale')
clf.fit(X_train, y_train)

# 預測
y_pred = clf.predict(X_test)

# 計算準確率
accuracy = accuracy_score(y_test, y_pred)

print(f"\nSVM 準確率: {accuracy*100:.2f}%")
print(f"\n這表示模型能夠正確辨識 {accuracy*100:.2f}% 的 SSVEP 頻率！")

## 7. 結果評估與視覺化

In [ ]:
# 繪製混淆矩陣
fig, ax = plt.subplots(figsize=(8, 6))

class_names = [f'{freq}Hz' for freq in target_freqs]
cm = confusion_matrix(y_test, y_pred)

# 正規化到 0-1
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# 繪製熱圖
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            ax=ax, cbar_kws={'label': 'Proportion'})

ax.set_title(f'Confusion Matrix\nAccuracy: {accuracy*100:.2f}%', 
            fontsize=13, fontweight='bold')
ax.set_ylabel('True Label', fontsize=11)
ax.set_xlabel('Predicted Label', fontsize=11)

plt.tight_layout()
plt.show()

print("圖說明: 混淆矩陣 (Confusion Matrix)")
print("對角線上的數值越高越好，代表預測正確")
print("非對角線的數值代表分類錯誤")

In [ ]:
# 詳細分類報告
print("="*60)
print("分類報告 (Classification Report)")
print("="*60)

print(classification_report(y_test, y_pred, 
                          target_names=class_names,
                          digits=4))

print("\n指標說明:")
print("  Precision (精確率): 預測為某類別中，實際正確的比例")
print("  Recall (召回率): 實際為某類別中，被正確預測的比例")
print("  F1-score: Precision 和 Recall 的調和平均")
print("  Support: 該類別在測試集中的樣本數")

## 8. 課程總結

### 我們學到了什麼？

1. **SSVEP 原理**: 大腦視覺皮層對特定頻率閃爍光的反應

2. **完整 Pipeline**:
   - 數據採集與視覺化
   - 訊號預處理（濾波、標準化）
   - 特徵提取（功率譜密度）
   - 分類器訓練與評估

3. **機器學習方法**:
   - 支持向量機 (SVM)
   - 訓練/測試集分割
   - 性能評估

### 實際應用
- 腦機介面 (BCI)
- 輔助溝通系統
- 遊戲控制
- 醫療復健

### 延伸學習
- 嘗試使用真實 EEG 數據集
- 探索深度學習方法 (CNN, LSTM)
- 優化特徵提取方法
- 實時 SSVEP 系統開發


## 9. 互動練習區

讓學員自己嘗試修改參數，觀察結果變化：

In [ ]:
# 練習 1: 改變訊噪比，觀察對分類性能的影響
# 提示: 修改 generate_ssvep_signal 函數中的 snr 參數（目前是 0.5）
# 試試看 snr=0.3 (更多雜訊) 或 snr=0.8 (更乾淨的訊號)

# 練習 2: 嘗試增加更多目標頻率
# 提示: 修改 target_freqs = [8, 10, 12, 15]
# 試試看加入 6Hz 和 14Hz: target_freqs = [6, 8, 10, 12, 14, 15]

# 練習 3: 調整濾波器參數，看看效果如何
# 提示: 修改 preprocess_signal 中的 lowcut=5 和 highcut=40
# 試試看 lowcut=8, highcut=30

# 練習 4: 嘗試使用不同數量的訓練數據
# 提示: 修改 n_trials = 40
# 試試看 n_trials=20 (較少數據) 或 n_trials=60 (較多數據)

print("請在上面的 code cells 中嘗試修改參數")
print("修改後重新執行所有 cells，觀察準確率的變化！")